# End goal is to find all papers than use MIMIC

## 1.1. Search PubMed

Begin by searching PubMed for citations to the MIMIC-II and MIMIC-III papers

In [12]:
import lcp.reuse as reuse

In [13]:
entrez_email = 'mimic-support@physionet.org'

In [14]:
reuse.helloworld()

'hello world'

In [15]:
results = reuse.search('mimic-iii',entrez_email)
print(results)

{'Count': '19', 'RetMax': '19', 'RetStart': '0', 'IdList': ['27219127', '28815150', '28460210', '28597703', '28548951', '28240612', '28442772', '28815112', '28423783', '27888170', '27824941', '27706278', '27694098', '26846781', '28328711', '27699003', '27489621', '27253619', '12403307'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'mimic-iii[All Fields]', 'Field': 'All Fields', 'Count': '19', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'mimic-iii[All Fields]'}


In [16]:
id_list = results['IdList']
papers = reuse.fetch_details(id_list,entrez_email)
print(papers)

{'PubmedArticle': [{'MedlineCitation': DictElement({'CitationSubset': ['IM'], 'OtherID': [StringElement('PMC4878278', attributes={'Source': 'NLM'})], 'OtherAbstract': [], 'KeywordList': [], 'SpaceFlightMission': [], 'GeneralNote': [], 'PMID': StringElement('27219127', attributes={'Version': '1'}), 'DateCreated': {'Year': '2016', 'Month': '05', 'Day': '25'}, 'DateCompleted': {'Year': '2016', 'Month': '12', 'Day': '16'}, 'DateRevised': {'Year': '2017', 'Month': '02', 'Day': '20'}, 'Article': DictElement({'ELocationID': [StringElement('10.1038/sdata.2016.35', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})], 'Language': ['eng'], 'ArticleDate': [DictElement({'Year': '2016', 'Month': '05', 'Day': '24'}, attributes={'DateType': 'Electronic'})], 'Journal': {'ISSN': StringElement('2052-4463', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'Volume': '3', 'PubDate': {'Year': '2016', 'Month': 'May', 'Day': '24'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'Scientific 